# Nomadic
## Framework to Framework language translation

This notebook is a demonstration of translating the `Flutter` framework to the `React Native` framework using the Encoder/Decoder architecture with the attention mechanism. We will train the model on `Flutter` and `React Native` code snippets to learn the mapping between the two frameworks. We will explore different methods for modeling the problem and evaluate the performance of the model.

### Vector Embedding for Repos

Word2Vec is a popular technique for learning vector representations of words. We can use the same technique to learn vector representations of code snippets. We can train a Word2Vec model on the code snippets of the repositories to learn the vector representations of the code programs. We can then use these vector representations as input to the Encoder/Decoder model.

In [14]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
import requests
from bs4 import BeautifulSoup

# Download NLTK resources (uncomment the following line if needed)
nltk.download('punkt')

def extract_text_from_package_links(package_links):
    descriptions = {}
    for link in package_links:
        print(link['href'])
        package_url = 'https://pub.dev' + link['href']
        package_response = requests.get(package_url)
        package_soup = BeautifulSoup(package_response.text, 'html.parser')
        all_text = package_soup.get_text()
        # remove unknown words from the text based on nltk
        tokens = word_tokenize(all_text)
        words = [word for word in tokens if word.isalpha()]
        all_text = ' '.join(words)
        descriptions[link['href'][10:]] = all_text
    return descriptions

def scrape_url(url) -> dict:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # find all package links with the class 'packages-title'
    packages = soup.find_all('h3', class_='packages-title')
    print(f"Found {len(packages)} packages")  # Debugging print statement
    package_links = [package.find('a') for package in packages]
    descriptions = extract_text_from_package_links(package_links)
    return descriptions

# Function to scrape package descriptions from pub.dev
def scrape_package_descriptions(sample_size=10):
    url = 'https://pub.dev/packages?q=sdk%3Aflutter'
    descriptions = scrape_url(url)
    for i in range(2, sample_size):
        url = f'https://pub.dev/packages?q=sdk%3Aflutter&page={i}'
        descriptions.update(scrape_url(url))
    return descriptions

# Scrape package descriptions
descriptions = scrape_package_descriptions()
descriptions

[nltk_data] Downloading package punkt to /Users/I589104/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Found 10 packages
/packages/http
/packages/shared_preferences
/packages/provider
/packages/image_picker
/packages/dio
/packages/cached_network_image
/packages/path_provider
/packages/permission_handler
/packages/flutter_bloc
/packages/geolocator
Found 10 packages
/packages/webview_flutter
/packages/go_router
/packages/flutter_native_splash
/packages/lottie
/packages/font_awesome_flutter
/packages/google_maps_flutter
/packages/share_plus
/packages/connectivity_plus
/packages/get_it
/packages/sqflite
Found 10 packages
/packages/shimmer
/packages/equatable
/packages/fl_chart
/packages/google_sign_in
/packages/uuid
/packages/package_info_plus
/packages/device_info_plus
/packages/logger
/packages/path
/packages/camera
Found 10 packages
/packages/crypto
/packages/smooth_page_indicator
/packages/flutter_riverpod
/packages/audioplayers
/packages/percent_indicator
/packages/location
/packages/flutter_rating_bar
/packages/flutter_lints
/packages/badges
/packages/sign_in_with_apple
Found 10 packa

{'http': 'http Dart packageSign for packagesPackage scoring and pub pointsFlutterUsing packagesDeveloping packages and pluginsPublishing a packageDartUsing packagesPublishing a Searching for packagesPackage scoring and pub pointsFlutter Using packagesDeveloping packages and pluginsPublishing a packageDart Using packagesPublishing a packagehttp http copied to clipboardPublished months ago composable API for HTTP ReadmeChangelogExampleInstallingVersionsScores A composable library for making HTTP requests This package contains a set of functions and classes that make it easy to consume HTTP resources It mobile desktop and browser and supports multiple implementations Using The easiest way to use this library is via the functions They allow you to make individual HTTP requests with minimal hassle import as http var url var response await url body print status print body print await Note Flutter applications may require additional configuration to make HTTP requests If you making multiple r

In [19]:
# Function to preprocess text data
def preprocess_text(text: str):
    tokens = word_tokenize(text.lower())
    # Add more preprocessing steps here if needed
    return tokens

# Main function to generate word embeddings
def generate_word_embeddings(descriptions):
    preprocessed_descriptions = [preprocess_text(desc) for desc in descriptions]
    model = Word2Vec(sentences=preprocessed_descriptions, vector_size=100, window=5, min_count=1, workers=4)
    return model

# Generate word embeddings
model = generate_word_embeddings(list(descriptions.values()))

# Test word similarity
print("Similarity between 'flutter' and 'dart':", model.wv.similarity('flutter', 'dart'))
print("Similarity between 'widget' and 'package':", model.wv.similarity('widget', 'package'))

# Save the model
model.save("models/flutter_package_embeddings.model")

Similarity between 'flutter' and 'dart': 0.8605835
Similarity between 'widget' and 'package': 0.5993923
